In [15]:
!pip3 install Cython
!pip3 install bert-score
!pip3 install tensorflow tensorflow_hub numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 MB 3.5 MB/s eta 0:00:0000:0100:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 2.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 2.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 2.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 3.5 MB/s eta 0:00:00a 0:00:01


In [ ]:
!pip install huggingface_hub[hf_xet]

from bert_score import score
generated = ["The quick brown fox jumps over the lazy dog"]

original_text = ["A fast brown  fox leaps over the sleepy dog"]
P, R, F1 = score(generated, original_text, lang="en", verbose=True)
print("Precision:", P)
print("Recall:", R)
print("F1 Score:", F1)


# Precision: tensor([0.9731])
# Recall: tensor([0.9626])
# F1 Score: tensor([0.9679])

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


100%|██████████| 1/1 [00:00<00:00,  6.28it/s]


computing greedy matching.


100%|██████████| 1/1 [00:00<00:00, 399.27it/s]


done in 0.16 seconds, 6.07 sentences/sec
Precision: tensor([0.9731])
Recall: tensor([0.9626])
F1 Score: tensor([0.9679])


In [26]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')  # 80MB model

def get_similarity(text1, text2):
    emb1 = model.encode(text1)
    emb2 = model.encode(text2)
    return np.dot(emb1, emb2) / (np.linalg.norm(emb1) * np.linalg.norm(emb2))

generated_text = "The death benefit payout/payment options for the Kotak e-Term policy are as follows:\n\n1. **Immediate Lump Sum Payment**: \n   - A lump sum payment of 10% of the Sum Assured on death shall be paid immediately at the time of claim settlement.\n\n2. **Level Recurring Payout**: \n   - 6% of the Sum Assured on death shall be payable immediately at the end of the first year from the date of death. \n   - The payout shall continue to increase by 10% every year (Simple Interest Rate) and these instalments shall be payable at the end of every year for 15 years starting from one year after the date of death.\n\n3. **Increasing Recurring Payout**: \n   - 6% of the Sum Assured on death shall be payable immediately at the end of every year for 15 years, with the first payment being made one year after the date of death of the life insured. \n   - If the nominee decides to receive the benefit in lump sum instead of recurring payouts, a discounted value of the outstanding payouts shall be paid as lump sum, calculated using a discounting rate of 6.0% p.a. (compounded).\n\n4. **Monthly Payment Option**: \n   - There is an option to receive the annual payment in monthly mode, where monthly payments shall be 8.22% of the Annual Payments. The first payment shall start from one month after the date of death of the life insured.\n\n5. **Accidental Death Benefit**: \n   - Under the \u2018Life Plus\u2019 option, the Accidental Death Benefit shall be payable in Lump sum irrespective of the above payout options selected."
reference_text = "1) Immediate: SAD paid in lump sum immediately and the policy ends; \\n2) Level Recurring: 10% of SA will be paid immediately on death and 6% of SA will be paid annually for 15 yrs, starting one year after death; \\nPayments can be monthly at 8.22% of the annual amount, starting 1 month after death; \\n3) Increasing Recurring: 10% of SA will be paid immediately on death and 6% of SA will be paid after 1 yr, increasing 10% yearly for 15 years; \\nPayments can be monthly at 8.22% of the annual amount, starting 1 month after death; \\n-Nominee can opt for a lump sum, discounted at 6%, except for Accidental Death Benefit"

gen_text = "Not Available"
ref_text = "Not available"
similarity_score = get_similarity(gen_text, ref_text)
print(f"Semantic similarity score: {similarity_score:.3f}")

Semantic similarity score: 1.000


In [20]:
import tensorflow as tf
import numpy as np
import tensorflow_hub as hub


model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

embed_fn = model.signatures['default']

# Define a function to compute semantic similarity
def get_similarity(text1, text2):
    # Convert to TensorFlow tensors
    inputs = tf.constant([text1, text2])
    
    # Get embeddings
    embeddings = embed_fn(inputs)["outputs"].numpy()
    
    # Compute cosine similarity
    similarity = np.inner(embeddings[0], embeddings[1])
    return similarity

# Test
generated_text = "A quick brown fox jumps over the lazy dog."
reference_text = "A fast brown fox leaps over a sleepy dog."

similarity_score = get_similarity(generated_text, reference_text)
print(f"Semantic similarity score: {similarity_score:.3f}") 

KeyError: 'default'